In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re

In [2]:
url = """https://insights.blackcoffer.com/how-to-protect-future-data-and-its-privacy-blackcoffer/"""

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
soup=BeautifulSoup(page.content, 'html.parser')

In [5]:
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
title

'How to protect future data and its privacy?'

In [6]:
content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
content

' Before the internet, information was in some ways restricted and more centralized. The only mediums of information were books, newspapers, and word of mouth, etc. But now with the advent of the internet and improvements to computer technology (Moore’s Law), information and data skyrocketed, and it has become this open-system, where information can be distributed to people without any kind of limits. SECURING YOUR DEVICES AND NETWORKS Encrypt your data Various publicly available tools have taken the rocket science out of encrypting (and decrypting) email and files. Data encryption isn’t just for technology geeks; modern tools make it possible for anyone to encrypt emails and other information. “Encryption used to be the sole province of geeks and mathematicians, but a lot has changed in recent years. In particular, various publicly available tools have taken the rocket science out of encrypting (and decrypting) email and files. GPG for Mail, for example, is an open-source plug-in for 

In [7]:
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
content

' Before the internet information was in some ways restricted and more centralized The only mediums of information were books newspapers and word of mouth etc But now with the advent of the internet and improvements to computer technology Moore’s Law information and data skyrocketed and it has become this opensystem where information can be distributed to people without any kind of limits SECURING YOUR DEVICES AND NETWORKS Encrypt your data Various publicly available tools have taken the rocket science out of encrypting and decrypting email and files Data encryption isn’t just for technology geeks modern tools make it possible for anyone to encrypt emails and other information “Encryption used to be the sole province of geeks and mathematicians but a lot has changed in recent years In particular various publicly available tools have taken the rocket science out of encrypting and decrypting email and files GPG for Mail for example is an opensource plugin for the Apple Mail program that 

In [8]:
text = content.split()
text

['Before',
 'the',
 'internet',
 'information',
 'was',
 'in',
 'some',
 'ways',
 'restricted',
 'and',
 'more',
 'centralized',
 'The',
 'only',
 'mediums',
 'of',
 'information',
 'were',
 'books',
 'newspapers',
 'and',
 'word',
 'of',
 'mouth',
 'etc',
 'But',
 'now',
 'with',
 'the',
 'advent',
 'of',
 'the',
 'internet',
 'and',
 'improvements',
 'to',
 'computer',
 'technology',
 'Moore’s',
 'Law',
 'information',
 'and',
 'data',
 'skyrocketed',
 'and',
 'it',
 'has',
 'become',
 'this',
 'opensystem',
 'where',
 'information',
 'can',
 'be',
 'distributed',
 'to',
 'people',
 'without',
 'any',
 'kind',
 'of',
 'limits',
 'SECURING',
 'YOUR',
 'DEVICES',
 'AND',
 'NETWORKS',
 'Encrypt',
 'your',
 'data',
 'Various',
 'publicly',
 'available',
 'tools',
 'have',
 'taken',
 'the',
 'rocket',
 'science',
 'out',
 'of',
 'encrypting',
 'and',
 'decrypting',
 'email',
 'and',
 'files',
 'Data',
 'encryption',
 'isn’t',
 'just',
 'for',
 'technology',
 'geeks',
 'modern',
 'tools',


In [9]:
len(text)

1976

In [10]:
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

56


In [11]:
#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)

64


In [12]:
filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])

# Adding Subjectivity & Polarity
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)

data

,url,title,content,filter_content,Positive_Score,Negative_Score,polarity,subjectivity
0,https://insights.blackcoffer.com/how-to-protec...,How to protect future data and its privacy?,Before the internet information was in some w...,Before the internet information was in some wa...,56,64,0.116277,0.465092


In [13]:
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)

Word average = 10242.0


In [14]:
import textstat

In [15]:
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print(FOG_INDEX)

793.7


In [16]:
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
AVG_NUMBER_OF_WORDS_PER_SENTENCE=print(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))

1976.0


In [17]:
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print(COMPLEX_WORDS)

3666


In [18]:
#Word Count
Word_Count=len(content)
print(Word_Count)

12222


In [19]:
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print(pcw)

29.995090819833088


In [20]:
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print(Average_Word_Length)

5.183198380566802


In [21]:
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
print(syllable_count/len(content.split()))

The AVG number of syllables in the word is: 
2.1300607287449393


In [2]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re
url = """https://insights.blackcoffer.com/how-to-protect-future-data-and-its-privacy-blackcoffer/"""
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
# title

content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
# print(content)
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
# print(content)
text = content.split()
# print(text)
len(text)
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)


filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
print(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
syllable = (syllable_count/len(content.split()))
print(syllable)

data = [[url,title,content,filter_content,Positive_score,Negative_score,AVG_SENTENCE_LENGTH,pcw,FOG_INDEX,
         AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score","Avg_Sentence_Length"
                               ,"Percentage_Complex_Word","Fog_Index"," AVG_NUMBER_OF_WORDS_PER_SENTENCE","COMPLEX_WORDS",
                               "Word_Count","syllable","Average_Word_Length"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data



56
64
Word average = 10242.0
FOG INDEX =  793.7
Average no of words per sentence
1976.0
Complex Words 3666
Word Count 12222
Percentage of Complex Words 29.995090819833088
Average Word per Length 5.183198380566802
The AVG number of syllables in the word is: 
2.1300607287449393


,url,title,content,filter_content,Positive_Score,Negative_Score,Avg_Sentence_Length,Percentage_Complex_Word,Fog_Index,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length,polarity,subjectivity
0,https://insights.blackcoffer.com/how-to-protec...,How to protect future data and its privacy?,Before the internet information was in some w...,Before the internet information was in some wa...,56,64,10242.0,29.995091,793.7,1976.0,3666,12222,2.130061,5.183198,0.116277,0.465092


In [3]:
data.to_csv(r'C:\Users\Om Bhandwalkar\Desktop\BlackCoffer Assignment\Output\url_47.csv')